In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=1)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((2,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 57357
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 60853
INFO:root:(('127.0.0.1', 57357)) Conexión aceptada con ('127.0.0.1', 60853)
INFO:root:(('127.0.0.1', 57357)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 57357)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 60853)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 57357)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 60853)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 60853)) Model initialized
INFO:root:(('127.0.0.1', 57357)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 57357)) Evaluating...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]

INFO:root:(('127.0.0.1', 60853)) Evaluating...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

INFO:root:(('127.0.0.1', 60853)) Evaluating...


TPU available: False, using: 0 TPU cores
GPU available: False, used: False
IPU available: False, using: 0 IPUs
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

INFO:root:(('127.0.0.1', 60853)) Broadcasting metrics to 1 clients.
INFO:root:(('127.0.0.1', 60853)) Finish!!.
SIN HACERR!!
True
INFO:root:(('127.0.0.1', 57357)) Broadcasting metrics to 1 clients.
INFO:root:(('127.0.0.1', 57357)) Finish!!.
SIN HACERR!!
True
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 57357 y desconectándose de 1 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 57357)
DEBUG:root:Closed connection: ('127.0.0.1', 60853)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 60853 y desconectándose de 0 nodos


In [ ]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)


In [2]:
#import threading
for thread in threading.enumerate(): 
    print(thread.name)

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
import sys

def get_thread_position(thread):
    frame = sys._current_frames().get(thread.ident, None)
    if frame:
        return frame.f_code.co_filename, frame.f_code.co_name, frame.f_code.co_firstlineno

get_thread_position(threading.enumerate()[-1])

('/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/node.py',
 '__on_round_finished',
 260)

In [1]:
from p2pfl.const import HEARTBEAT_FREC, SOCKET_TIEMOUT
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
from test.fixtures import two_nodes, four_nodes


import threading
for thread in threading.enumerate(): 
    print(thread.name)
    
n1 = Node(MLP(),MnistFederatedDM())
n2 = Node(MLP(),MnistFederatedDM())
print("\n")
def test_bad_binary_model():

    n1.start()
    n2.start()

    n1.connect_to(n2.host,n2.port)
    time.sleep(0.1) 

    # Start Learning
    n1.set_start_learning(rounds=2,epochs=0)
 
    # Adding noise to the buffer
    for _ in range(200):
        n1.neightboors[0].param_bufffer += "noise".encode("utf-8")
    
    while not n1.round is None and not n2.round is None:
        time.sleep(0.1)
        
    n1.stop()
    n2.stop()
    
test_bad_binary_model()

time.sleep(10)

for thread in threading.enumerate(): 
    print(thread.name)
    
print("\n")



MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2
Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000


INFO:root:Nodo a la escucha en 127.0.0.1 54998
INFO:root:Nodo a la escucha en 127.0.0.1 54999
INFO:root:(('127.0.0.1', 54999)) Conexión aceptada con ('127.0.0.1', 54998)
INFO:root:(('127.0.0.1', 54998)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 54998)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 54999)) Initialicing Model Weights
INFO:root:(('127.0.0.1', 54998)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 54998)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1', 54998)) Training...
INFO:root:(('127.0.0.1', 54999)) Model initialized
INFO:root:(('127.0.0.1', 54998)) Model added (1/2) from ('127.0.0.1', 54998)
INFO:root:(('127.0.0.1', 54998)) Broadcasting model to 1 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 54999)) Model added (1/2) from ('127.0.0.1', 54

In [4]:

for thread in threading.enumerate(): 
    print(thread.name)
    

MainThread
IOPub
Heartbeat
Thread-3
Thread-4
Control
IPythonHistorySavingThread
Thread-2


In [6]:
print(n1.round)

None


In [4]:
from p2pfl.settings import Settings

Settings.AGREGATION_TIEMOUT = 1

In [5]:
"0.1545318067073822".isdecimal()

False

In [4]:
message = ['METRICS', '0', '2.301456928253174', '0.13700607419013977']
print(int(message[1]), float(message[2]), float(message[3]))

0 2.301456928253174 0.13700607419013977
